In [3]:
import io
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer, tokenizer_from_json
from keras_preprocessing.sequence import pad_sequences

In [19]:
with open('tags.txt', encoding='utf8') as file_in:
    X = []
    Y = []
    for line in file_in:
        Y.append(line.strip().split(' ')[-1])
        X.append(' '.join(line.strip().split(' ')[:-1]))
Y = pd.get_dummies(pd.DataFrame(np.asarray(Y).astype(np.float)).rename(columns={0: 'label'})['label'], prefix='label_').values

In [21]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)


In [22]:
vocab_size = 2000
embed_size = 50
max_len = 100

tokenizer = Tokenizer(num_words=vocab_size,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      oov_token="<OOV>")

tokenizer.fit_on_texts(x_train)
training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(
    training_sequences, maxlen=max_len, truncating='post')


tokenizer_json = tokenizer.to_json()
with io.open('aira_tokenizer_en.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [24]:
inputs = tf.keras.Input(shape=(None,), dtype="int32")
x = tf.keras.layers.Embedding(input_dim=vocab_size,
                              output_dim=embed_size,
                              input_length=max_len)(inputs)
# 2 bidirectional LSTMs
x = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
# classifier
outputs = tf.keras.layers.Dense(142, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
history = model.fit(training_padded,
                    y_train,
                    epochs=50,
                    batch_size=5)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, maxlen=100, truncating='post')

test_loss_score, test_acc_score = model.evaluate(test_padded, y_test)

print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')
model.save('pre_trained_aira')

Epoch 1/50
311/311 [==============================] - 18s 41ms/step - loss: 4.9497 - categorical_accuracy: 0.0097
Epoch 2/50
311/311 [==============================] - 13s 41ms/step - loss: 4.7821 - categorical_accuracy: 0.0200
Epoch 3/50
311/311 [==============================] - 13s 41ms/step - loss: 4.6054 - categorical_accuracy: 0.0290
Epoch 4/50
311/311 [==============================] - 13s 41ms/step - loss: 4.3566 - categorical_accuracy: 0.0644
Epoch 5/50
311/311 [==============================] - 13s 41ms/step - loss: 4.2138 - categorical_accuracy: 0.0753
Epoch 6/50
311/311 [==============================] - 13s 41ms/step - loss: 4.1339 - categorical_accuracy: 0.0985
Epoch 7/50
311/311 [==============================] - 13s 42ms/step - loss: 4.0044 - categorical_accuracy: 0.1198
Epoch 8/50
311/311 [==============================] - 13s 41ms/step - loss: 3.8205 - categorical_accuracy: 0.1487
Epoch 9/50
311/311 [==============================] - 18s 59ms/step - loss: 3.6990 - cat